# **EARTHQUAKE DAMAGE PREDICTION**

https://www.drivendata.org/competitions/57/nepal-earthquake/page/136/

* Andrea Morales Garzón `andreamgmg@correo.ugr.es`
* Ithiel Piñero Darias `ithiel@correo.ugr.es`
* Paula Villa Martín `pvilla@correo.ugr.es`
* Antonio Manjavacas Lucas `manjavacas@correo.ugr.es`

Basándonos en factores relacionados con la localización de los edificios y su construcción, el objetivo de este trabajo será predecir el nivel de daño provocado por el terremoto Gorkha de 2015 sobre edificios en Nepal.

Los datos fueron recopilados por medio de encuestas realizadas por Kathmandu Living Labs y la Oficina Central de Estadística, dependiente de la Comisión Nacional de Planificación de la Secretaría de Nepal. Esta encuesta es uno de los mayores conjuntos de datos posteriores a un desastre jamás reunidos, y 
contiene información valiosa sobre los efectos de los terremotos, las condiciones de los hogares y estadísticas socioeconómicas y demográficas.

Trataremos de predecir la variable ordinal `damage_grade`, que representa el nivel de daño provocado sobre los edificios afectados por el terremoto:

* `damage_grade` = 1 representa un daño bajo;
* `damage_grade` = 2 representa un daño medio;
* `damage_grade` = 3 representa una destrucción del edificio casi completa.


In [1]:
install.packages('tidyverse')
install.packages('NoiseFiltersR')
install.packages('caret')

install.packages('MLmetrics')
install.packages('UBL')
install.packages('mltools')
install.packages('data.table')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# **Preprocesamiento**

Carga de datos:

In [2]:
# **1. Carga de datos
library(tidyverse)
TRAIN_VALUES_ID = '15ykpkKIJNKlEXQQ3taspRjUZ2sJN5zS_'
TRAIN_LABELS_ID = '1nrNVfj9NmNvwPhXuucUBYhh-FmODCCBK'
TEST_VALUES_ID = '1_GpX1sh7XkJLm-kyOpcObzXICW5Z-tb_'

load_file <- function(id) {
  read_csv(sprintf('https://docs.google.com/uc?id=%s&export=download', id), col_types=cols())
}
train_values <- load_file(TRAIN_VALUES_ID)
train_labels <- load_file(TRAIN_LABELS_ID)
test_values <- load_file(TEST_VALUES_ID)

buiding_id_test <- test_values$building_id

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.5     ✔ dplyr   1.0.3
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Conversión de variables:

In [3]:
# Conversión de las columnas con datos categóricos a factores
cols_to_factor <- c(2:4, 9:15, 27)
train_values[cols_to_factor] <- lapply(train_values[cols_to_factor], factor)
test_values[cols_to_factor] <- lapply(test_values[cols_to_factor], factor)

train_values <- merge(x = train_values, y = train_labels, by = 'building_id')
head(train_values)

# la variable de la clase tiene que ser factor
train_labels$damage_grade <- as.factor(train_labels$damage_grade)


#preprocesamiento
is_categorical_var <- sapply(train_values, is.factor)
cat_vars <- train_values[, is_categorical_var]

names(cat_vars)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,⋯,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,30,266,1224,1,25,5,2,t,r,⋯,0,0,0,0,0,0,0,0,0,2
2,8,17,409,12182,2,0,13,7,t,r,⋯,0,0,0,0,0,0,0,0,0,3
3,12,17,716,7056,2,5,12,6,o,r,⋯,0,0,0,0,0,0,0,0,0,3
4,16,4,651,105,2,80,5,4,n,r,⋯,0,0,0,0,0,0,0,0,0,2
5,17,3,1387,3909,5,40,5,10,t,r,⋯,0,0,0,0,0,0,0,0,0,2
6,25,26,1132,6645,2,0,6,6,t,w,⋯,0,0,0,0,0,0,0,0,0,1


[1] "geo_level_1_id"         "geo_level_2_id"         "geo_level_3_id"        
 [4] "land_surface_condition" "foundation_type"        "roof_type"             
 [7] "ground_floor_type"      "other_floor_type"       "position"              
[10] "plan_configuration"     "legal_ownership_status"

Agrupamiento de categorías:

In [4]:
## Agrupamiento de categorías
# En este apartado agrupamos las categorías con igual comportamiento/distribución para el nivel de daño:

group_label <- function(x, label1, label2, new_label) {
  x <- sub(label1, new_label, x)
  x <- sub(label2, new_label, x)
}

group_cat <- function(data, var, label1, label2, grouped_label) {
  as.factor(sapply(data[, var], group_label, label1, label2, grouped_label))
}

# Train
train_values$plan_configuration <- group_cat(train_values, 'plan_configuration', 'a', 'c', 'a+c')
train_values$plan_configuration <- group_cat(train_values, 'plan_configuration', 'd', 'n', 'd+n')
train_values$plan_configuration <- group_cat(train_values, 'plan_configuration', 'd+n', 'q', 'd+n+q')
train_values$foundation_type <- group_cat(train_values, 'foundation_type', 'u', 'w', 'u+w')
train_values$roof_type <- group_cat(train_values, 'roof_type', 'n', 'q', 'n+q')
train_values$ground_floor_type <- group_cat(train_values, 'ground_floor_type', 'f', 'x', 'f+x')
train_values$other_floor_type <- group_cat(train_values, 'other_floor_type', 'q', 'x', 'q+x')

# Test
test_values$plan_configuration <- group_cat(test_values, 'plan_configuration', 'a', 'c', 'a+c')
test_values$plan_configuration <- group_cat(test_values, 'plan_configuration', 'd', 'n', 'd+n')
test_values$plan_configuration <- group_cat(test_values, 'plan_configuration', 'd+n', 'q', 'd+n+q')
test_values$foundation_type <- group_cat(test_values, 'foundation_type', 'u', 'w', 'u+w')
test_values$roof_type <- group_cat(test_values, 'roof_type', 'n', 'q', 'n+q')
test_values$ground_floor_type <- group_cat(test_values, 'ground_floor_type', 'f', 'x', 'f+x')
test_values$other_floor_type <- group_cat(test_values, 'other_floor_type', 'q', 'x', 'q+x')




In [5]:
train_values %>% select(starts_with('has_superstructure')) %>% names
group_superstructure <- function(data) {
  # Division in: non-robust, robust
  data <- data %>% mutate(superstructure = 
                            ifelse(has_superstructure_adobe_mud == 1 | has_superstructure_mud_mortar_brick == 1 |
                                     has_superstructure_mud_mortar_stone == 1 | has_superstructure_stone_flag == 1, "non-robust", "robust")
  )
  
  # Convert to factor and delete grouped variables
  data$superstructure <- as.factor(data$superstructure)
  data <- data %>% select(-starts_with('has_superstructure'))
  
}

train_values <- group_superstructure(train_values)
test_values <- group_superstructure(test_values)

# ------------------------------------------------------------------------------

[1] "has_superstructure_adobe_mud"          
 [2] "has_superstructure_mud_mortar_stone"   
 [3] "has_superstructure_stone_flag"         
 [4] "has_superstructure_cement_mortar_stone"
 [5] "has_superstructure_mud_mortar_brick"   
 [6] "has_superstructure_cement_mortar_brick"
 [7] "has_superstructure_timber"             
 [8] "has_superstructure_bamboo"             
 [9] "has_superstructure_rc_non_engineered"  
[10] "has_superstructure_rc_engineered"      
[11] "has_superstructure_other"

Agrupamiento de `superstructure`:

* **ROBUST**: `cement-mortar-stone`, `cement-mortar-brick`,`timber`,`bamboo`,`rc-non-engineered`,`rc-engineered`y`other`.
* **NON-ROBUST**: `adobe-mud`, `mud-mortar-brick`,`mud-mortar-stone`y`stone-flag`.

Agrupación de variables categóricas: uso secundario:

* **HOUSING**: `hotel`, `rental`.
* **GOVERNANCE**: `gov_office`, `institution`.
* **AGRICULTURE**: `agriculture`.
* **SERVICES**: `police`, `school`, `health_post`.
* **INDUSTRY**: `industry`.
* **NONE**.

In [6]:
train_values %>% select(starts_with('has_secondary')) %>% names
group_secondary_use <- function(data) {
  # Division by use: housing, governance, agriculture, services, industry, none
  data <- data %>% mutate(secondary_use =
                            ifelse(has_secondary_use_hotel == 1 | has_secondary_use_rental == 1, 'housing',
                                   ifelse(has_secondary_use_gov_office == 1 | has_secondary_use_institution == 1, 'governance',
                                          ifelse(has_secondary_use_agriculture == 1, 'agriculture',
                                                 ifelse(has_secondary_use_use_police == 1 | has_secondary_use_school == 1 | has_secondary_use_health_post == 1, 'services',
                                                        ifelse(has_secondary_use_industry == 1, 'industry', 'none')))))
  )
  
  # Convert to factor and delete grouped variables
  data$secondary_use <- as.factor(data$secondary_use)
  data <- data %>% select(-starts_with('has_secondary'))
}

train_values <- group_secondary_use(train_values)
test_values <- group_secondary_use(test_values)

[1] "has_secondary_use"             "has_secondary_use_agriculture"
 [3] "has_secondary_use_hotel"       "has_secondary_use_rental"     
 [5] "has_secondary_use_institution" "has_secondary_use_school"     
 [7] "has_secondary_use_industry"    "has_secondary_use_health_post"
 [9] "has_secondary_use_gov_office"  "has_secondary_use_use_police" 
[11] "has_secondary_use_other"

In [7]:

datos <- data.frame(train_values)
datos$damage_grade <- as.factor(datos$damage_grade)
datos <- datos[,-1]

datos$geo_level_1_id <- as.factor(datos$geo_level_1_id)
datos$geo_level_2_id <- as.numeric(datos$geo_level_2_id)
datos$geo_level_3_id <- as.numeric(datos$geo_level_3_id)

# **Modelo: Regresión logística**

## **a) Modelo 2 vs ALL**

In [8]:

train_data_2_vs_all <- datos
train_data_2_vs_all$damage_grade <- as.factor(ifelse(train_data_2_vs_all$damage_grade == 2, 1,0))

logit2 <- glm(as.factor(damage_grade) ~ . ,family=binomial(link="logit"), data=train_data_2_vs_all)

Ahora podemos predecir sobre el propio conjunto de train, para ver su comportamiento sobre el conjunto de entrenamiento.

In [9]:
pred <- predict(logit2,train_data_2_vs_all[,-39])
cats.pred = rep("0", dim(train_data_2_vs_all)[1])
# el umbral de decisión lo dejamos de momento en 0.5, aunque después lo variaremos.
cats.pred[pred > 0.50] = "1"
print(table(cats.pred, train_data_2_vs_all$damage_grade))
mean(cats.pred == train_data_2_vs_all$damage_grade)

         
cats.pred     0     1
        0 80368 63020
        1 31974 85239


[1] 0.6354811

## **b) Modelo 1 vs 3**

In [10]:
train_data_1_vs_3 <- datos[which(datos$damage_grade != 2),]
train_data_1_vs_3$damage_grade <- as.factor(ifelse(train_data_1_vs_3$damage_grade == 1, 1,0))
#1VS3
table(train_data_1_vs_3$damage_grade)

# entrenar modelo
logit <- glm(as.factor(damage_grade) ~ .,family=binomial(link="logit"), data=train_data_1_vs_3)
# visualización del modelo
logit


    0     1 
87218 25124 


Call:  glm(formula = as.factor(damage_grade) ~ ., family = binomial(link = "logit"), 
    data = train_data_1_vs_3)

Coefficients:
            (Intercept)          geo_level_1_id1          geo_level_1_id2  
              2.169e+00                9.801e-01               -3.383e-01  
        geo_level_1_id3          geo_level_1_id4          geo_level_1_id5  
             -3.487e+00               -1.096e+00                1.264e+00  
        geo_level_1_id6          geo_level_1_id7          geo_level_1_id8  
             -2.042e+00               -2.289e+00               -3.446e+00  
        geo_level_1_id9         geo_level_1_id10         geo_level_1_id11  
             -1.700e+00               -2.522e+00               -3.188e+00  
       geo_level_1_id12         geo_level_1_id13         geo_level_1_id14  
             -5.734e-01                1.014e+00                3.261e-01  
       geo_level_1_id15         geo_level_1_id16         geo_level_1_id17  
             -5.120e-01         

In [11]:


# predecir sobre el conjunto de train para ver cómo se comporta
pred <- predict(logit,train_data_1_vs_3[,-39])
cats.pred = rep("0", dim(train_data_1_vs_3)[1])
cats.pred[pred > .50] = "1"
table(cats.pred, train_data_1_vs_3$damage_grade)
mean(cats.pred == train_data_1_vs_3$damage_grade)


         
cats.pred     0     1
        0 85160  8691
        1  2058 16433

[1] 0.904319

# **Resultados**

In [12]:
# predicción en test
test_values$geo_level_1_id <- as.factor(test_values$geo_level_1_id)
test_values$geo_level_2_id <- as.numeric(test_values$geo_level_2_id)
test_values$geo_level_3_id <- as.numeric(test_values$geo_level_3_id)

# primero predecimos qué elementos son de la clase 2. 
pred_2_vs_all <- predict(logit2,test_values)
pred_2_vs_all.values = rep("0", dim(test_values)[1])

pred_2_vs_all.values[pred_2_vs_all > 0.35] = "2" # umbral de decisión fijado a 35
table(pred_2_vs_all.values)

# guardamos para la submision los elementos que se predicen a la clase 2
res_2_vs_all <- as.data.frame(cbind(buiding_id_test, pred_2_vs_all.values)) # x1=2, x0=otro
colnames(res_2_vs_all) <- c("building_id", "damage_grade")
head(res_2_vs_all)

# le pasamos el clasificador 1vs3 a las variables que se han predicho como "no" clase 2.
test_data_1_vs_3 <- test_values[which(pred_2_vs_all.values == "0"),]
building_id_test_1_vs_3 <- buiding_id_test[which(pred_2_vs_all.values == "0")]

# predecir para distinguir entre 1 y 3
pred_1_vs_3 <- predict(logit,test_data_1_vs_3)
pred_1_vs_3.values = rep("3", dim(test_data_1_vs_3)[1])
pred_1_vs_3.values[pred_1_vs_3 > 0.65] = "1" #0.65
table(pred_1_vs_3.values)

res_etiquetas_1_3 <- as.data.frame(cbind(building_id_test_1_vs_3, pred_1_vs_3.values)) 
head(res_etiquetas_1_3)
head(pred_1_vs_3.values)

# res_etiquetas_1_3$pred_1_vs_3.values[which(res_etiquetas_1_3$pred_1_vs_3.values == 2)] <- 3
head(res_etiquetas_1_3)
head(pred_1_vs_3.values)
colnames(res_etiquetas_1_3) <- c("building_id", "damage_grade")
head(res_etiquetas_1_3)


res_2_vs_all <- res_2_vs_all[which(pred_2_vs_all.values == "2"),] 
res_final <- rbind(res_2_vs_all,res_etiquetas_1_3)
table(res_final$damage_grade)

test_values.final <- load_file(TEST_VALUES_ID)
head(test_values.final)
library(plyr)
test_values.final <- join(x = test_values.final, y = res_final)
head(test_values.final)
table(res_final$damage_grade)


pred_2_vs_all.values
    0     2 
40671 46197 

,building_id,damage_grade
,<chr>,<chr>
1,300051,0
2,99355,2
3,890251,2
4,745817,2
5,421793,0
6,871976,2


pred_1_vs_3.values
    1     3 
 7184 33487 

,building_id_test_1_vs_3,pred_1_vs_3.values
,<chr>,<chr>
1,300051,3
2,421793,3
3,691228,1
4,896100,3
5,652685,3
6,590834,3


[1] "3" "3" "1" "3" "3" "3"

...


    1     2     3 
 7184 46197 33487 

building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,⋯,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
300051,17,596,11307,3,20,7,6,t,r,⋯,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,⋯,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,⋯,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,⋯,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,⋯,0,0,0,0,0,0,0,0,0,0
871976,22,170,4029,1,55,4,3,t,r,⋯,0,0,0,0,0,0,0,0,0,0


------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from ‘package:purrr’:

    compact


Joining by: building_id



,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,⋯,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,300051,17,596,11307,3,20,7,6,t,r,⋯,0,0,0,0,0,0,0,0,0,3
2,99355,6,141,11987,2,25,13,5,t,r,⋯,0,0,0,0,0,0,0,0,0,2
3,890251,22,19,10044,2,5,4,5,t,r,⋯,0,0,0,0,0,0,0,0,0,2
4,745817,26,39,633,1,0,19,3,t,r,⋯,0,1,0,0,0,0,0,0,0,2
5,421793,17,289,7970,3,15,8,7,t,r,⋯,0,0,0,0,0,0,0,0,0,3
6,871976,22,170,4029,1,55,4,3,t,r,⋯,0,0,0,0,0,0,0,0,0,2



    1     2     3 
 7184 46197 33487 

In [13]:
write.csv(test_values.final[,c("building_id", "damage_grade")],"submit.csv", row.names = FALSE)

Extra añadido:
Función FSelector utilizada para las pruebas, que da muy buenos resultados pero los resultados salen un poco inferiores que la configuración final. 

In [14]:
# library(FSelector)
# 
# fEvaluacion <- function(dataset = NULL,
#                         classIndex = 0,
#                         k = 0) {
#   function(subset) {
#     # genera valores aleatorios (uniforme) para cada
#     # muestra del conjunto de datos
#     splits <- runif(nrow(dataset))
#     # tratamiento de cada una de las particiones. Para cada
#     # valor de particion se aplica la funcion que se define
#     # a continuacion
#     results <- sapply(1:k, function(i) {
#       print("entra en el forward")
#       # se determina el indice de las muestras para test
#       # (aproximadamente una fraccion 1/k de las muestras
#       # del conjunto de datos)
#       test.idx <- (splits >= ((i - 1) / k) & (splits < (i / k)))
#       # todas las demas muestras seran para training
#       train.idx <- !test.idx
#       # se seleccionan las muestras en si
#       test <- dataset[test.idx,]
#       train <- dataset[train.idx,]
#       # aprende el modelo sobre el conjunto de entrenamiento
#       
#       className <- names(dataset)[classIndex]
#       
#       # logit2 <- glm(as.factor(damage_grade) ~ as.factor(geo_level_1_id) + poly(count_floors_pre_eq,3) + poly(age,3) + poly(area_percentage,3) 
#       #               + poly(area_percentage,3) + foundation_type  +  has_superstructure_cement_mortar_stone
#       #               ,family=binomial(link="logit"), data=train_data_2_vs_all)
#       
#       # pred <- predict(logit2,train_data_2_vs_all[,-39])
#       # cats.pred = rep("0", dim(train_data_2_vs_all)[1])
#       # cats.pred[pred > 0.5] = "1"
#       
#       lm.fit <- glm(as.simple.formula(subset, className),family=binomial(link="logit"), data = train)
#       # obtener porcentaje de aciertos
#       pred <- predict(lm.fit,test)
#       cats.pred = rep("0", dim(test)[1])
#       cats.pred[pred > .50] = "1"
#       print(table(cats.pred),test$damage_grade)
#       
#       aciertos <- mean(cats.pred == test$damage_grade)
#       # devuelve la tasa de aciertos
#       aciertos
#     })
#     # se muestra el subconjunto y la media de resultados
#     # y se devuelve la media de los resultados (un resultado
#     # por particion)
#     print(subset)
#     print(mean(results))
#     mean(results)
#   }
# }